In [ ]:
import os
from googletrans import Translator
from moviepy.editor import VideoFileClip, concatenate_videoclips, AudioFileClip, ImageSequenceClip, CompositeAudioClip
import soundfile as sf

EXTERNAL = True

if EXTERNAL:
  %store -r WIDTH
  %store -r HEIGHT
  %store -r TEXT
  %store -r VISUALIZE
  %store -r SAVE_FOLDER
  %store -r PROJECT_NAME
  %store -r SHOW_OUTPUT
  %store -r IMG_NUMBER
  %store -r THUMBNAIL
  %store -r SKIP_VIDEO
  %store -r MERGE_VIDEO
  %store -r TEXT_VOICE_GEN
  %store -r LANGUAGES
  %store -r GEN_VIDEO
  %store -r INSERT_BACKGROUND
  %store -r RECORD_FRAME_INTER_AFTER
  %store -r RECORD_FRAME_INTER_HD_AFTER
else:
  WIDTH = 768
  HEIGHT = 768
  VISUALIZE = True  
  SAVE_FOLDER = "D:\\Deletar\\p_gen"
  PROJECT_NAME = "THE FELLOWSHIP OF THE RING"
  STYLES_FOLDER = ".\\styles"
  SHOW_OUTPUT = True
  IMG_NUMBER = 2
  SKIP_VIDEO = []
  MERGE_VIDEO = False
  TEXT_VOICE_GEN = ['A','A','A','A','A','A','A','A','A','A','A','A','A','A','A','A','A','A','A']
  LANGUAGES = ['pt']
  GEN_VIDEO = True
  INSERT_BACKGROUND = False
  RECORD_FRAME_INTER_AFTER = 0
  RECORD_FRAME_INTER_HD_AFTER = 0

translator = Translator()
project_folder = f"{SAVE_FOLDER}//{PROJECT_NAME}"

In [ ]:
def create_srt(text, initial_time, final_time, indx, filename):
  initial_mil = int(round(initial_time % 1,3)*1000)
  final_mil = int(round(final_time % 1,3)*1000)
  initial_time = int(initial_time)
  final_time = int(final_time)

  minutes, seconds = divmod(initial_time, 60)
  hours, minutes = divmod(minutes, 60)
  initial_time = f'{hours:02d}:{minutes:02d}:{seconds:02d},{initial_mil}'

  minutes, seconds = divmod(final_time, 60)
  hours, minutes = divmod(minutes, 60)
  final_time = f'{hours:02d}:{minutes:02d}:{seconds:02d},{final_mil}'

  print(filename)

  with open(f"{filename}//en.srt" , 'w', encoding='utf-8') as f:
    f.write(f'{int(indx)+1}\n')
    f.write(f'{initial_time} --> {final_time}\n')
    f.write(f'{text}\n')
    f.write('\n')
  
  for lang in LANGUAGES:
    transl = translator.translate(text, src='en', dest=lang).text
    with open(f"{filename}//{lang}.srt" , 'w', encoding='utf-8') as f:
      f.write(f'{int(indx)+1}\n')
      f.write(f'{initial_time} --> {final_time}\n')
      f.write(f'{transl}\n')
      f.write('\n')

def merge_srt_files(filenames, output_filename):
    with open(output_filename, 'w', encoding='utf-8') as outfile:
        for i, filename in enumerate(filenames):
            with open(filename, 'r', encoding='utf-8') as infile:
                for line in infile:
                    outfile.write(line)
                if i < len(filenames) - 1:
                    outfile.write('\n')

In [ ]:
filenames = []
filenames_other = {}
for lan in LANGUAGES:
    filenames_other[lan] = []
start_time = 0  
cont = -1
durations = []

for folder_name in os.listdir(os.path.join(f"{project_folder}\\audio\\")):
    for filename in os.listdir(os.path.join(f"{project_folder}\\audio\\{folder_name}")):
      duration = sf.SoundFile(os.path.join(f"{project_folder}\\audio\\{folder_name}\\{filename}"))      
      durations.append(duration.frames / duration.samplerate)

for folder in os.listdir(f"{project_folder}//video_sound_hd"):
        for filename in os.listdir(f"{project_folder}//video_sound_hd//{folder}"):
            if (filename[-3:] == 'mp4'):

                cont = cont + 1      
                addr_video = f"{project_folder}//video_sound_hd//{folder}//{filename}"
                video = VideoFileClip(addr_video)   
                video_duration = video.duration                 
                """ original_audio = video.audio
                original_audio_duration = round(original_audio.duration,3) """
                original_audio_duration = durations[cont]
                create_srt(TEXT_VOICE_GEN[str(cont)], start_time + 0.5, start_time + original_audio_duration, cont, 
                filename=f"{project_folder}//video_sound_hd//{cont:05d}")
                start_time = round(start_time + video_duration,3)            


for folder in os.listdir(f"{project_folder}//video_sound_hd"):
        for filename in os.listdir(f"{project_folder}//video_sound_hd//{folder}"):
            if (filename[-6:] == 'en.srt'):                    
                filenames.append(f"{project_folder}//video_sound_hd//{folder}//{filename}")
            elif(filename[-3:] == 'srt'):
                lan = filename[-6:-4]
                filenames_other[lan] = filenames_other[lan] + [f"{project_folder}//video_sound_hd//{folder}//{filename}"]
    
if len(filenames) > 0:
    merge_srt_files(filenames, f"{project_folder}//full//full_hd.srt")
    for lang_mod in filenames_other:
        merge_srt_files(filenames_other[lang_mod], f"{project_folder}//full//{lang_mod}.srt")